# 合并ksrpc+frpc两阻塞代码于单Notebook

In [1]:
def run_ksrpc():
    print("启动ksrpc服务")
    
    from ksrpc import config
    config.PORT = 8000

    from ksrpc.app import simple_
    simple_.main()

In [2]:
def run_frpc():
    print("启动frpc服务")
    
    import subprocess
    import threading

    # 读取流的函数
    def read_stream(stream, prefix):
        for line in stream:
            print(f"[{prefix}] {line.strip()}")

    # 启动进程
    process = subprocess.Popen(
        ['./frpc', '-c', './frpc.toml'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True,
        bufsize=1   # 行缓冲，实时输出
    )

    # 创建并启动线程
    threads = [
        threading.Thread(target=read_stream, args=(process.stdout, "STDOUT")),
        threading.Thread(target=read_stream, args=(process.stderr, "STDERR"))
    ]
    
    for thread in threads:
        thread.daemon = True  # 设置为守护线程，主线程结束时自动退出
        thread.start()

    try:
        # 等待进程结束
        return_code = process.wait()
    except KeyboardInterrupt:
        print("\n用户中断，终止进程...")
        process.terminate()
        return_code = process.wait()
    finally:
        # 等待线程结束（守护线程会在进程结束后自动退出）
        for thread in threads:
            thread.join(timeout=1)  # 设置超时防止永久阻塞

    # 检查返回码
    if return_code != 0:
        print(f"命令执行出错，返回码: {return_code}")

In [3]:
def background_clear():
    print("启动定时清屏")
    
    import time
    from IPython.display import clear_output
    
    while True:
        time.sleep(15)
        clear_output(wait=True)

In [ ]:
import asyncio
import concurrent.futures

async def main():
    # 获取事件循环
    try:
        loop = asyncio.get_running_loop()  # Python 3.7+
    except AttributeError:
        loop = asyncio.get_event_loop()    # Python 3.6 及更早版本
    
    # 并行执行两个函数
    with concurrent.futures.ThreadPoolExecutor() as pool:
        await asyncio.gather(
            loop.run_in_executor(pool, run_ksrpc),
            loop.run_in_executor(pool, run_frpc),
            loop.run_in_executor(pool, background_clear),
        )

# Jupyter 中直接运行异步函数
await main()